In [3]:
import numpy as np
import pandas as pd

from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate


import matplotlib.pyplot as plt
%matplotlib inline

ModuleNotFoundError: No module named 'surprise'

1.As in your previous assignments, compare the accuracy of at least two recommender system algorithms against your offline data.

2.Implement support for at least one business or user experience goal such as increasedserendipity, novelty, or diversity.

3.Compare and report on any change in accuracy before and after you’ve made the change in #2. 

4.As part of your textual conclusion, discuss one or more additional experiments that could be performed and/or metrics that could be evaluated only if online evaluation was possible.  Also, briefly propose how you would design a reasonable online evaluation environment.

In [2]:
# load movielens data

ratings = pd.read_csv('https://raw.githubusercontent.com/mjdacs/data612/master/project_4/ml-latest-small/ratings.csv')
movies = pd.read_csv('https://raw.githubusercontent.com/mjdacs/data612/master/project_4/ml-latest-small/movies.csv', index_col='movieId')


In [3]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
ratings.shape

(100836, 4)

In [5]:
movies.head()

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [6]:
movies.shape

(9742, 2)

In [8]:
ratings_movies_merge = ratings.merge(movies, on='movieId').drop(columns=['timestamp', 'genres'])
ratings_movies_merge.head()

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)


In [9]:
ratings_movies_merge.shape

(100836, 4)

In [10]:
m = ratings_movies_merge.pivot(index='userId', columns='movieId', values='rating')
m.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [128]:
df = m.dropna(axis=0, thresh=25).dropna(axis=1, thresh=75)
df.shape

(547, 229)

In [120]:
col_count = 0
for i in range(0, m.shape[1]):
    if m.iloc[:,i].count() >= 60:
        col_count += 1
print(col_count)

335


In [79]:
df.iloc[:,0].count()

215

In [78]:
df.iloc[0, :].count()

232

In [122]:
row_count = 0
for i in range(1, m.shape[0]+1):
    if m.loc[i, :].count() >= 25:
        row_count += 1
print(row_count)

547


In [129]:
df.head()

movieId,1,2,6,10,16,19,21,32,34,39,...,51662,54286,58559,59315,60069,68157,68954,69122,72998,79132
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.5,NaN,NaN,4.5,NaN,NaN,NaN,4.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [139]:
col = df.count(axis=0)
col.min()

75

In [140]:
# build training and test sets
items_to_keep = 15
rating_threshold = 3
n_eval = 1

train, test = train_test_split(df, test_size=0.2, random_state=11)

In [141]:
train.shape

(437, 229)

In [142]:
test.shape

(110, 229)